In [1]:
!pip install transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00


In [2]:
## Import libraries
import torch
import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
from trl import RewardTrainer, SFTTrainer
from datasets import Dataset
import json
import pandas as pd
from transformers import Trainer, TrainingArguments

In [3]:
!wget https://huggingface.co/datasets/CarperAI/openai_summarize_comparisons/resolve/main/data/test-00000-of-00001-0845e2eec675b16a.parquet
# after that rename to test.parquet

--2023-09-06 12:36:39--  https://huggingface.co/datasets/CarperAI/openai_summarize_comparisons/resolve/main/data/test-00000-of-00001-0845e2eec675b16a.parquet
Resolving huggingface.co (huggingface.co)... 18.172.134.24, 18.172.134.4, 18.172.134.88, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/02/84/02842250b699bbde4e980e9206f5045df542f9ca374595cc86b8d57189641ff5/a8c5d73113a2c7119faacd44e43959f74e5b59e181f3da57caa918c907ac6620?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27test-00000-of-00001-0845e2eec675b16a.parquet%3B+filename%3D%22test-00000-of-00001-0845e2eec675b16a.parquet%22%3B&Expires=1694262999&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5NDI2Mjk5OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8wMi84NC8wMjg0MjI1MGI2OTliYmRlNGU5ODBlOTIwNmY1MDQ1ZGY1NDJmOWNhMzc0NTk1

### Dataset for Training : https://huggingface.co/datasets/CarperAI/openai_summarize_tldr

### Dataset for Testing : https://huggingface.co/datasets/CarperAI/openai_summarize_comparisons

In [4]:
Model_name = "bigcode/tiny_starcoder_py"
Data_path = "test.parquet"

In [7]:
# Read the file
df  = pd.read_parquet(Data_path)
df = df[:10]
raw_dataset = Dataset.from_pandas(df)
raw_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 10
})

### Tokenize the dataset to feed in the model

In [8]:
tokenizer = AutoTokenizer.from_pretrained(Model_name)
model = AutoModelForCausalLM.from_pretrained(Model_name)

### Tokenize the dataset to feed in the model

In [9]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
def formatting_func(examples):
    kwargs = {"padding": "max_length",
              "truncation": True,
              "max_length": 256,
              "return_tensors": "pt"
              }

    # Prepend the prompt and a line break to the original_response and response-1 fields.
    prompt_plus_chosen_response = examples["prompt"] + "\n" + examples["chosen"]
    prompt_plus_rejected_response = examples["prompt"] + "\n" + examples["rejected"]

    # Then tokenize these modified fields.
    tokens_chosen = tokenizer.encode_plus(prompt_plus_chosen_response, **kwargs)
    tokens_rejected = tokenizer.encode_plus(prompt_plus_rejected_response, **kwargs)

    return {
        "input_ids_chosen": tokens_chosen["input_ids"][0], "attention_mask_chosen": tokens_chosen["attention_mask"][0],
        "input_ids_rejected": tokens_rejected["input_ids"][0], "attention_mask_rejected": tokens_rejected["attention_mask"][0]
    }

### Map the function to data

In [10]:
formatted_dataset = raw_dataset.map(formatting_func)
formatted_dataset = formatted_dataset.train_test_split()

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [11]:
model.config

GPTBigCodeConfig {
  "_name_or_path": "bigcode/tiny_starcoder_py",
  "activation_function": "gelu_pytorch_tanh",
  "architectures": [
    "GPTBigCodeForCausalLM"
  ],
  "attention_softmax_in_fp32": true,
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "inference_runner": 0,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "max_batch_size": null,
  "max_sequence_length": null,
  "model_type": "gpt_bigcode",
  "multi_query": true,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": 3072,
  "n_layer": 20,
  "n_positions": 8192,
  "pad_key_length": true,
  "pre_allocate_kv_cache": false,
  "resid_pdrop": 0.1,
  "scale_attention_softmax_in_fp32": true,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.33.0",
  "use_cache": true,
  "validate_runner_input": tr

In [12]:
### Loading the TRL reward trainer and training the trainer
training_args = TrainingArguments(
        output_dir="tinystarcoder-rlhf-model",
        num_train_epochs=1,
        logging_steps=10,
        gradient_accumulation_steps=1,
        save_strategy="steps",
        evaluation_strategy="steps",
        per_device_train_batch_size=2,
        per_device_eval_batch_size=1,
        eval_accumulation_steps=1,
        eval_steps=500,
        save_steps=500,
        warmup_steps=100,
        logging_dir="./logs",
        learning_rate=1e-5,
        save_total_limit=1,
        no_cuda=True
    )

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1257: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [13]:
trainer = RewardTrainer(model=model,
                        tokenizer=tokenizer,
                        train_dataset=formatted_dataset['train'],
                        eval_dataset=formatted_dataset['test'],
                        args= training_args
                        )
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/reward_trainer.py:124: UserWarning: When using RewardDataCollatorWithPadding, you should set `max_length` in the RewardTrainer's init it will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/reward_trainer.py:138: UserWarning: When using RewardDataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2436: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length,

Step,Training Loss,Validation Loss


TrainOutput(global_step=4, training_loss=0.7674217224121094, metrics={'train_runtime': 101.4459, 'train_samples_per_second': 0.069, 'train_steps_per_second': 0.039, 'total_flos': 0.0, 'train_loss': 0.7674217224121094, 'epoch': 1.0})

In [15]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2436: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/reward_trainer.py:214: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3571.)
  logits = torch.stack(logits).mean(dim=2).softmax(dim=0).T


{'eval_loss': 0.6931466460227966,
 'eval_accuracy': 0.0,
 'eval_runtime': 22.5484,
 'eval_samples_per_second': 0.133,
 'eval_steps_per_second': 0.133,
 'epoch': 1.0}

In [17]:
from huggingface_hub import login
login()

In [18]:
trainer.push_to_hub("RLHF model of StarCoder")

pytorch_model.bin:   0%|          | 0.00/657M [00:00<?, ?B/s]

'https://huggingface.co/ashishpatel26/tinystarcoder-rlhf-model/tree/main/'

In [20]:
trainer.save_model("tinystarcoder-rlhf-model")

### Policy Model

In [ ]:
help(trainer.push_to_hub)

In [21]:
## inference the model
rm_model = AutoModelForCausalLM.from_pretrained("tinystarcoder-rlhf-model")
tokenizer = AutoTokenizer.from_pretrained("tinystarcoder-rlhf-model")

In [22]:
def get_score(model, tokenizer, prompt, response):

    instructions = tokenizer.encode_plus(prompt,
                                       response,
                                       padding="max_length",
                                       max_length=256,
                                       return_tensors="pt",
                                        truncation=True)
    with torch.no_grad():
        outputs = model(**instructions)

    logits = outputs[0]

    return logits

In [23]:
# usage with prompt
prompt = df.iloc[0]["prompt"]
example_prefered_response = df.iloc[0]["chosen"]
example_unprefered_response = df.iloc[0]["rejected"]

In [24]:
loss1 = get_score(model, tokenizer, prompt, example_prefered_response)
loss2= get_score(model, tokenizer, prompt, example_unprefered_response)

In [25]:
from torch import nn
loss = -nn.functional.logsigmoid(loss1 - loss2).mean()

In [26]:
tokenizer.decode(torch.max(loss1, axis=-1).indices[0])

'_DDIT_\n           "r/#: Relationship RelationshipRelationship0]0ARCH\nlsriend\n2//M]\n [\n [\n the was to the [\n a friends to\n\n: https [lfriend [ me am a aumb. my19 minutes.\n\n"""I updated:** girlfriend was through my Facebook..** I my my friends.**.** my  of dayslf.\n\n** went dding for my message personirl** I had to makeellpping my my my past** but I was a in\n\n** have ali a of ago she  tolirt with my girl. and she had my about my.. me few of gir.1viously). was\'t find that was).\n\n** was it for  twoirl and the had my Facebook. the   she wasand historyirl) was was March,\n to to find that were flited. I a g.. f.ing on her.\nirirl wass; I1 girirllfriend is 19 months. through my Facebook.. my permission. I it messages. her.lirty with me coupleirl.\n found up with me after she through more with\n'

### Policy Model

In [54]:
import torch
import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
from trl import RewardTrainer, SFTTrainer
from datasets import Dataset
import json
import pandas as pd
from transformers import Trainer, TrainingArguments
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model

In [55]:
##model path
MODEL_PATH = "bigcode/tiny_starcoder_py"
DATA_PATH = "test.parquet"

In [56]:
df = pd.read_parquet(DATA_PATH)
df = df[:1000]
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 1000
})

In [59]:
from transformers import AutoTokenizer, pipeline

In [62]:
rf_model_path = "tinystarcoder-rlhf-model"
starcoder_model = AutoModelForCausalLMWithValueHead.from_pretrained(rf_model_path)
starcoder_model_ref = AutoModelForCausalLMWithValueHead.from_pretrained(rf_model_path)
starcoder_tokenizer = AutoTokenizer.from_pretrained(rf_model_path)

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

In [63]:
txt_in_len = 5
txt_out_len = 32
seed = 1

dataset = dataset.map(
    lambda x: {"input_ids": tokenizer.encode(" " + x["chosen"], return_tensors="pt", truncation=True, padding="max_length", max_length=32)[0]},
    batched=False,
)
dataset = dataset.map(lambda x: {"query": tokenizer.decode(x["input_ids"])}, batched=False)
dataset = dataset[:20480]
from datasets import Dataset

dataset = Dataset.from_dict(dataset)
dataset.set_format("pytorch")

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [64]:
dataset

Dataset({
    features: ['review', 'chosen', 'rejected', 'input_ids', 'query'],
    num_rows: 1000
})

In [98]:
sentiment_pipe_kwargs = {"top_k": None, "function_to_apply": "none"}

config = PPOConfig(
    model_name=MODEL_PATH, steps=51200, learning_rate=1.41e-5, remove_unused_columns=True
)

txt_in_len = 5
txt_out_len = 20
seed = 1

In [99]:
import torch
optimizer = torch.optim.SGD(starcoder_model.parameters(), lr=config.learning_rate)
ppo_trainer = PPOTrainer(config, starcoder_model, starcoder_model, starcoder_tokenizer, dataset=dataset, data_collator=collator, optimizer=optimizer)

In [100]:
# for i in ppo_trainer.dataloader:
#   print(i)

In [101]:
ctrl_str = ["[negative]", "[positive]"]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # this should be handled by accelerate
ctrl_tokens = dict((s, starcoder_tokenizer.encode(s, return_tensors="pt").squeeze().to(device)) for s in ctrl_str)

In [102]:
def pos_logit_to_reward(logit, task):
    """
    Take the positive sentiment logit and scale it for the task.
        task [negative]: reward = -logit
        task [neutral]: reward = -2*abs(logit)+4
        task [positive]: reward = logit
    """
    for i in range(len(logit)):
        if task[i] == "[negative]":
            logit[i] = -logit[i]
        elif task[i] == "[positive]":
            pass
        else:
            raise ValueError("task has to be in [0, 1, 2]!")
    return logit

In [103]:
pos_logit_to_reward(torch.Tensor([4, 4]), ctrl_str)

tensor([-4.,  4.])

In [104]:
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": starcoder_tokenizer.eos_token_id,
    "max_new_tokens": 32,
    "eos_token_id": -1,
}

In [105]:
def get_score(model, tokenizer, responses):
    positive_logist = []
    for i in responses:
        instructions = tokenizer.encode_plus(
                                           i,
                                           padding="max_length",
                                           max_length=32,
                                           return_tensors="pt")
        with torch.no_grad():
            outputs = model(**instructions)

        logits = outputs[0].mean()
        positive_logist.append(logits)

    return positive_logist


In [113]:
# responses =["ashish is a goo", "heelow how are you", "__IT_\nr/\n: r RelationshipRelationship]]0]\nlsriend\n2//M]\n [ [ a\n the was to the [. a friends to\n\n:\n [lfriend [ me have a aried in his19 minutes.\n\nWhat Modified:** girlfriend was through the Facebook.. I my my friends.**** my  of lf**\n\n** was d1ing for my few personirl** I had for findoolpping my my the future** but I was that in\n\n** have ali  of to she  tolirt my me girl. and she found my about my.. me few of gir.1viously). was\'t find her was).\n\n** was it about my twoirl and the had  Facebook. the  and she gand historyirl) was in April,\n to, find, were flirted. I a messages.. f.ing on her.\n girlM\n; I1 girirllfriend and the19 months. to my Facebook.. my permission. she her messages. my.lirty with my fewirl.\n found her with me. I through more with\n"]
# get_score(starcoder_model, tokenizer, responses)

In [115]:
# from random import choices
# from tqdm import tqdm
# import time
# import numpy as np

# for epoch in range(1):
#     for batch in tqdm(ppo_trainer.dataloader):
#         (logs, game_data,) = (
#             dict(),
#             dict(),
#         )

#         print(ctrl_str)
#         #### prepend a random control token
#         task_list = choices(ctrl_str, k=config.batch_size)
#         game_data["query"] = [t + q for t, q in zip(task_list, batch["query"])]
#         query_tensors = [torch.cat((ctrl_tokens[t], input_ids)) for t, input_ids in zip(task_list, batch["input_ids"])]

#         #### get response from gpt2
#         response_tensors = []
#         for query in query_tensors:
#             response = ppo_trainer.generate(query, **generation_kwargs)
#             response_tensors.append(response.squeeze()[-txt_out_len:])
# #         print(response_tensors)
#         game_data["response"] = [starcoder_tokenizer.decode(r.squeeze()) for r in response_tensors]

#         #### sentiment analysis
#         texts = [q + r for q, r in zip(batch["query"], game_data["response"])]
#         logits = get_score(starcoder_model,starcoder_tokenizer, texts)
#         rewards = pos_logit_to_reward(logits, task_list)

#         #### Run PPO training
#         t = time.time()
#         stats = ppo_trainer.step(query_tensors, response_tensors, rewards)

#         for cs in ctrl_str:
#             key = "env/reward_" + cs.strip("[]")
#             stats[key] = np.mean([r.cpu().numpy() for r, t in zip(rewards, task_list) if t == cs])
#         ppo_trainer.log_stats(stats, game_data, rewards)

In [109]:
###saving the model
starcoder_model.save_pretrained("StarCoderTinyrhlfmodel/")
starcoder_tokenizer.save_pretrained("StarCoderTinyrhlfmodel/")

('StarCoderTinyrhlfmodel/tokenizer_config.json',
 'StarCoderTinyrhlfmodel/special_tokens_map.json',
 'StarCoderTinyrhlfmodel/vocab.json',
 'StarCoderTinyrhlfmodel/merges.txt',
 'StarCoderTinyrhlfmodel/added_tokens.json',
 'StarCoderTinyrhlfmodel/tokenizer.json')

In [110]:
from transformers import pipeline, set_seed
model_path = "StarCoderTinyrhlfmodel/"
set_seed(42)
pipe = pipeline("text-generation",model=model_path, tokenizer=model_path, max_length=30, num_return_sequences=5)

Some weights of the model checkpoint at StarCoderTinyrhlfmodel/ were not used when initializing GPTBigCodeForCausalLM: ['v_head.summary.weight', 'v_head.summary.bias']
- This IS expected if you are initializing GPTBigCodeForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPTBigCodeForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
